In [1]:
import pandas as pd
import numpy as np

Load in `.csv` file that came from `stage-p2x`

In [2]:
df = pd.read_csv('runs_with_eval_loss_and_params.csv')

Load in data from MolNet finetuning

In [3]:
import s3fs
import os

In [4]:
fs = s3fs.S3FileSystem()

In [5]:
import os
import json

In [1]:
model_bucket = ""

In [6]:
cloud_dir = f's3://{model_bucket}/chemberta/mtr_pretraining_10M_20210819/molnet_mtr_10M_ft_20210819/'

In [7]:
def get_dataframes(cloud_dir):
    run_dirs = fs.ls(cloud_dir)
    data_avg = []
    df_all = pd.DataFrame()
    for rd in run_dirs:
        run_name = os.path.basename(os.path.normpath(rd))
        # go one level down to get the molnet task
        molnet_task_data_avg = {}
        molnet_task_data_all = {}
        for molnet_task_dir in fs.ls(rd):
            molnet_task_name = os.path.basename(os.path.normpath(molnet_task_dir))
            results_dir = os.path.join(molnet_task_dir, "results/")
            for subset in ["valid", "test"]:
                with fs.open(os.path.join(results_dir, subset, "metrics.json")) as f:
                    metrics = json.load(f)
                # pick first item to get the keys
                metric_names = list(list(metrics.items())[0][1].keys())
                metric_res = {mn: [] for mn in metric_names}
                for seed, res in metrics.items():
                    for mn, mres in res.items():
                        if mn == "pearsonr":
                            metric_res[mn].append(mres[0])
                        else:
                            metric_res[mn].append(mres)
                molnet_task_data_all.update({f"{molnet_task_name}_{subset}_{mn}": metric_res[mn] for mn in metric_names})
                average_metrics = {f"{molnet_task_name}_{subset}_{mn}_mean": np.mean(metric_res[mn]) for mn in metric_names}
                std_metrics = {f"{molnet_task_name}_{subset}_{mn}_std": np.std(metric_res[mn]) for mn in metric_names}
                molnet_task_data_avg.update({**average_metrics, **std_metrics})
        molnet_task_data_all.update({"run_name": [run_name]*5})
        df_all = df_all.append(pd.DataFrame(molnet_task_data_all))
        data_avg.append({"run_name": run_name, **molnet_task_data_avg})

    df_avg = pd.DataFrame(data_avg)
    return df_all, df_avg

In [8]:
df_all, df_avg = get_dataframes(cloud_dir)

In [9]:
df_all

,bace_classification_valid_roc_auc_score,bace_classification_valid_average_precision_score,bace_classification_test_roc_auc_score,bace_classification_test_average_precision_score,bace_regression_valid_pearsonr,bace_regression_valid_rmse,bace_regression_test_pearsonr,bace_regression_test_rmse,bbbp_valid_roc_auc_score,bbbp_valid_average_precision_score,...,delaney_test_rmse,lipo_valid_pearsonr,lipo_valid_rmse,lipo_test_pearsonr,lipo_test_rmse,tox21_valid_roc_auc_score,tox21_valid_average_precision_score,tox21_test_roc_auc_score,tox21_test_average_precision_score,run_name
0,0.538024,0.613755,0.728261,0.808682,0.131829,0.484727,0.722628,1.097644,0.969915,0.966416,...,0.430124,0.722625,0.697898,0.680087,0.670627,0.768531,0.484334,0.738588,0.417091,run_0
1,0.537846,0.613705,0.728261,0.808682,0.141178,0.486687,0.711637,1.086831,0.970497,0.966397,...,0.429206,0.722392,0.698363,0.679760,0.671285,0.780156,0.492071,0.740300,0.419935,run_0
2,0.538024,0.613755,0.728261,0.808682,0.132674,0.487636,0.719533,1.100704,0.969915,0.967760,...,0.430106,0.723193,0.697662,0.681314,0.669751,0.769561,0.487122,0.741083,0.428582,run_0
3,0.538024,0.613755,0.728261,0.808682,0.137811,0.487263,0.713470,1.088938,0.971759,0.970452,...,0.427102,0.722993,0.698298,0.679727,0.671142,0.777625,0.494367,0.741083,0.420956,run_0
4,0.538024,0.613755,0.728442,0.808801,0.141110,0.486964,0.712663,1.088569,0.970303,0.965588,...,0.432635,0.723781,0.696310,0.680215,0.670578,0.768016,0.493614,0.739077,0.415587,run_0
0,0.638415,0.663049,0.810507,0.827066,0.189216,0.522431,0.796469,1.051947,0.971079,0.969825,...,0.406910,0.748160,0.680665,0.718023,0.659613,0.789207,0.470662,0.760458,0.404072,run_1
1,0.636638,0.662242,0.802899,0.820672,0.188512,0.526209,0.796557,1.059410,0.971953,0.970524,...,0.407189,0.769539,0.656364,0.732854,0.649797,0.769689,0.462807,0.770194,0.399088,run_1
2,0.632196,0.659250,0.800000,0.820283,0.190720,0.524959,0.796926,1.045800,0.972244,0.970459,...,0.401759,0.748592,0.687884,0.718813,0.670387,0.754290,0.480408,0.766623,0.401440,run_1
3,0.637882,0.662238,0.811232,0.827947,0.189093,0.524700,0.796415,1.057534,0.972050,0.970454,...,0.401579,0.760329,0.668827,0.727620,0.654072,0.758022,0.467323,0.767699,0.399345,run_1
4,0.634151,0.658300,0.804167,0.823830,0.191925,0.523195,0.797127,1.044148,0.972923,0.971726,...,0.407807,0.767214,0.658575,0.732291,0.648803,0.760896,0.453929,0.776310,0.400191,run_1


In [10]:
df_avg

,run_name,bace_classification_valid_roc_auc_score_mean,bace_classification_valid_average_precision_score_mean,bace_classification_valid_roc_auc_score_std,bace_classification_valid_average_precision_score_std,bace_classification_test_roc_auc_score_mean,bace_classification_test_average_precision_score_mean,bace_classification_test_roc_auc_score_std,bace_classification_test_average_precision_score_std,bace_regression_valid_pearsonr_mean,...,lipo_test_pearsonr_std,lipo_test_rmse_std,tox21_valid_roc_auc_score_mean,tox21_valid_average_precision_score_mean,tox21_valid_roc_auc_score_std,tox21_valid_average_precision_score_std,tox21_test_roc_auc_score_mean,tox21_test_average_precision_score_mean,tox21_test_roc_auc_score_std,tox21_test_average_precision_score_std
0,run_0,0.537989,0.613745,0.000071,0.000020,0.728297,0.808705,0.000072,0.000048,0.136920,...,0.000578,0.000540,0.772778,0.490302,0.005079,0.003908,0.740026,0.420430,0.001027,0.004507
1,run_1,0.635856,0.661016,0.002348,0.001878,0.805761,0.823960,0.004390,0.003159,0.189893,...,0.006394,0.007907,0.766421,0.467026,0.012475,0.008737,0.768257,0.400827,0.005146,0.001818
2,run_38,0.510483,0.551737,0.000000,0.000000,0.698333,0.714754,0.000072,0.000242,0.174632,...,0.010102,0.001651,0.753826,0.442080,0.002990,0.007599,0.824111,0.451679,0.004665,0.003686
3,run_39,0.681663,0.700822,0.000775,0.002206,0.833116,0.849589,0.001391,0.000953,0.173427,...,0.003468,0.004154,0.746774,0.434499,0.001141,0.010602,0.816517,0.415473,0.006292,0.014972
4,run_45,0.624058,0.688055,0.000379,0.000320,0.734565,0.801115,0.000136,0.000271,0.224502,...,0.002370,0.005729,0.772358,0.464837,0.001415,0.002154,0.782661,0.413957,0.001878,0.001396


In [11]:
combined_avg_df = pd.merge(left=df, right=df_avg, on='run_name')
# combined_avg_df['run_name'] = combined_avg_df['run_name'].apply(lambda x: f"mlm_{x}")

combined_all_df = pd.merge(left=df, right=df_all, on='run_name')
# combined_all_df['run_name'] = combined_all_df['run_name'].apply(lambda x: f"mlm_{x}")

In [12]:
combined_avg_df

,run_name,min_eval_loss,hidden_size,attention_probs_dropout_prob,hidden_dropout_prob,intermediate_size,num_attention_heads,num_hidden_layers,learning_rate,per_device_train_batch_size,...,lipo_test_pearsonr_std,lipo_test_rmse_std,tox21_valid_roc_auc_score_mean,tox21_valid_average_precision_score_mean,tox21_valid_roc_auc_score_std,tox21_valid_average_precision_score_std,tox21_test_roc_auc_score_mean,tox21_test_average_precision_score_mean,tox21_test_roc_auc_score_std,tox21_test_average_precision_score_std
0,run_38,0.286096,126,0.109,0.279,456,3,2,0.000021,25,...,0.010102,0.001651,0.753826,0.442080,0.002990,0.007599,0.824111,0.451679,0.004665,0.003686
1,run_39,0.204584,209,0.176,0.128,3968,11,3,0.000002,25,...,0.003468,0.004154,0.746774,0.434499,0.001141,0.010602,0.816517,0.415473,0.006292,0.014972
2,run_45,0.046702,384,0.109,0.144,464,12,3,0.000141,25,...,0.002370,0.005729,0.772358,0.464837,0.001415,0.002154,0.782661,0.413957,0.001878,0.001396
3,run_1,0.136169,407,0.156,0.295,9044,11,2,0.000005,25,...,0.006394,0.007907,0.766421,0.467026,0.012475,0.008737,0.768257,0.400827,0.005146,0.001818
4,run_0,0.074779,264,0.224,0.122,4888,8,3,0.000396,25,...,0.000578,0.000540,0.772778,0.490302,0.005079,0.003908,0.740026,0.420430,0.001027,0.004507


In [13]:
combined_all_df

,run_name,min_eval_loss,hidden_size,attention_probs_dropout_prob,hidden_dropout_prob,intermediate_size,num_attention_heads,num_hidden_layers,learning_rate,per_device_train_batch_size,...,delaney_test_pearsonr,delaney_test_rmse,lipo_valid_pearsonr,lipo_valid_rmse,lipo_test_pearsonr,lipo_test_rmse,tox21_valid_roc_auc_score,tox21_valid_average_precision_score,tox21_test_roc_auc_score,tox21_test_average_precision_score
0,run_38,0.286096,126,0.109,0.279,456,3,2,0.000021,25,...,0.885124,0.480149,0.603597,0.810276,0.569769,0.782754,0.758622,0.443152,0.823768,0.452073
1,run_38,0.286096,126,0.109,0.279,456,3,2,0.000021,25,...,0.874281,0.500813,0.624136,0.803858,0.592573,0.781701,0.749271,0.430675,0.818386,0.454400
2,run_38,0.286096,126,0.109,0.279,456,3,2,0.000021,25,...,0.869565,0.508930,0.610882,0.807713,0.574930,0.785854,0.752960,0.436813,0.820148,0.445391
3,run_38,0.286096,126,0.109,0.279,456,3,2,0.000021,25,...,0.862123,0.523273,0.617961,0.805920,0.583830,0.784608,0.754204,0.451888,0.831352,0.456078
4,run_38,0.286096,126,0.109,0.279,456,3,2,0.000021,25,...,0.857925,0.530797,0.628011,0.806330,0.596377,0.785752,0.754075,0.447873,0.826900,0.450453
5,run_39,0.204584,209,0.176,0.128,3968,11,3,0.000002,25,...,0.900703,0.449770,0.768226,0.648632,0.703411,0.661425,0.745196,0.437009,0.810852,0.435366
6,run_39,0.204584,209,0.176,0.128,3968,11,3,0.000002,25,...,0.901392,0.449247,0.749706,0.670351,0.701203,0.660973,0.748370,0.418800,0.820343,0.412874
7,run_39,0.204584,209,0.176,0.128,3968,11,3,0.000002,25,...,0.893759,0.465103,0.749826,0.667276,0.696345,0.667063,0.747641,0.451820,0.820833,0.403296
8,run_39,0.204584,209,0.176,0.128,3968,11,3,0.000002,25,...,0.900301,0.448992,0.765977,0.649216,0.707039,0.656524,0.746740,0.433024,0.823328,0.429604
9,run_39,0.204584,209,0.176,0.128,3968,11,3,0.000002,25,...,0.901878,0.446191,0.767808,0.648892,0.702398,0.667650,0.745925,0.431841,0.807231,0.396227


In [14]:
combined_avg_df.to_csv('ft_results_combined.csv', index=False)
combined_all_df.to_csv('ft_results_all_seeds.csv', index=False)